In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
!pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 KB 5.0 MB/s eta 0:00:00


In [3]:
import keras_tuner as kt

# Load data

In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [5]:
img_train.shape, img_test.shape

((60000, 28, 28), (10000, 28, 28))

In [6]:
# normalization
img_train = img_train.astype('float32') / 255.
img_test = img_test.astype('float32') / 255.

# img_train.dtype, img_test.dtype

# Create Model

In [7]:
def model_builder(hp):
  model = keras.models.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(units=10))

  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
      loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=['accuracy']
  )

  return model

## Create tuner model

In [8]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [9]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [10]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 50s]
val_accuracy: 0.8569166660308838

Best val_accuracy So Far: 0.8901666402816772
Total elapsed time: 00h 12m 51s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 96 and the optimal learning rate for the optimizer
is 0.001.



In [11]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5264 - accuracy: 0.8139 - val_loss: 0.4440 - val_accuracy: 0.8364
Epoch 2/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3914 - accuracy: 0.8585 - val_loss: 0.3816 - val_accuracy: 0.8644
Epoch 3/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3517 - accuracy: 0.8728 - val_loss: 0.3721 - val_accuracy: 0.8673
Epoch 4/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3279 - accuracy: 0.8806 - val_loss: 0.3654 - val_accuracy: 0.8692
Epoch 5/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3063 - accuracy: 0.8876 - val_loss: 0.3319 - val_accuracy: 0.8811
Epoch 6/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2946 - accuracy: 0.8899 - val_loss: 0.3410 - val_accuracy: 0.8792
Epoch 7/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2784 - accuracy: 0.8964 - val_loss: 0.3225 - val_accuracy:

In [12]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/25
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5283 - accuracy: 0.8142 - val_loss: 0.4076 - val_accuracy: 0.8541
Epoch 2/25
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3908 - accuracy: 0.8606 - val_loss: 0.4005 - val_accuracy: 0.8570
Epoch 3/25
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3515 - accuracy: 0.8726 - val_loss: 0.3660 - val_accuracy: 0.8667
Epoch 4/25
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3266 - accuracy: 0.8816 - val_loss: 0.3319 - val_accuracy: 0.8792
Epoch 5/25
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3070 - accuracy: 0.8880 - val_loss: 0.3697 - val_accuracy: 0.8718
Epoch 6/25
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2926 - accuracy: 0.8912 - val_loss: 0.3484 - val_accuracy: 0.8774
Epoch 7/25
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2799 - accuracy: 0.8965 - val_loss: 0.3220 - val_accuracy:

In [13]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 2ms/step - loss: 0.3882 - accuracy: 0.8850
[test loss, test accuracy]: [0.3881872892379761, 0.8849999904632568]
